In [94]:
import os
import requests
import io

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt

# Data Import

In [95]:
url_folder='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master'
cube_files=['cube_JAN2019-JUN2019.csv']
url=url_folder+'/'+cube_files[0]
s=requests.get(url).content
Data=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=';')

other_files=['SCNTXL.csv']

In [96]:
# url_folder='https://raw.githubusercontent.com/dkremlg/Booking-Curves/master'

# cube_files=['cube_JAN2018-MAR2018.csv','cube_APR2018-JUN2018.csv','cube_JUL2018-SEP2018.csv','cube_OCT2018-DEC2018.csv',
# 'cube_JAN2019-MAR2019.csv','cube_APR2019-JUN2019.csv','cube_JUL2019-SEP2019.csv']

# other_files=['SCNTXL.csv']

# url=url_folder+'/'+cube_files[0]
# s=requests.get(url).content
# Data=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=';')

# for f in cube_files:
#     url=url_folder+'/'+f
#     s=requests.get(url).content
#     Data_intermediate=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=';')
#     Data=pd.concat([Data,Data_intermediate])

### Extract day and month

In [97]:
Data=Data.dropna(how='any')
Data.index=range(Data.shape[0])

Data=Data.rename(columns={'Extraction Day': 'IssueDate','Calendar day': 'DepDate','Sched Airport Depart': 'SegFrom',
'Sched Airport Arriva': 'SegTo','Pax total': 'NumPax','Flight Number': 'FltNum'})
Data['FltNum']=Data['FltNum'].apply(lambda x: str(str(x).split('-')[1]))
Data['NumPax']=Data['NumPax'].apply(lambda x: float(str(x).split(' ')[0]))

Data['DepDayMonth']=Data['DepDate'].apply(lambda x: str(x).split('/')[0]+'/'+str(x).split('/')[1])
Data['DepMonth']=Data['DepDate'].apply(lambda x: str(x).split('/')[1]+'/'+str(x).split('/')[2])

### Map destination combination to inbound or outbound

In [98]:
Map_AirportsToDirection=DataFrame([['SCN','TXL'],['TXL','SCN'],['O','I']]).transpose()
Map_AirportsToDirection.columns=['SegFrom','SegTo','Direction']
Data=Data.merge(Map_AirportsToDirection,on=['SegTo','SegFrom'])

### Map flight number to departure time

In [99]:
# IMPORT FLIGHT NUMBER/DEPARTURE TIME MAPPING TABLE

url=url_folder+'/'+other_files[0]
s=requests.get(url).content
Map_Flt_To_DepTime=pd.read_csv(io.StringIO(s.decode('utf-8')),sep=',')
Map_Flt_To_DepTime['FltNum']=Map_Flt_To_DepTime['FltNum'].astype('str')

Map_Flt_To_DepTime['DepDate']=Map_Flt_To_DepTime['DepDate'].apply(lambda x: '/'.join([x.split('-')[2],x.split('-')[1],x.split('-')[0]]) if '-' in x else x)
Map_Flt_To_DepTime['DepDay']=Map_Flt_To_DepTime['DepDate'].apply(lambda x: x.split('/')[0])
Map_Flt_To_DepTime['DepMonth']=Map_Flt_To_DepTime['DepDate'].apply(lambda x: x.split('/')[1])
Map_Flt_To_DepTime['DepYear']=Map_Flt_To_DepTime['DepDate'].apply(lambda x: x.split('/')[2])
Map_Flt_To_DepTime['DepDay']=Map_Flt_To_DepTime['DepDay'].apply(lambda x: '0'+x if len(x)==1 else x)
Map_Flt_To_DepTime['DepMonth']=Map_Flt_To_DepTime['DepMonth'].apply(lambda x: '0'+x if len(x)==1 else x)
Map_Flt_To_DepTime['DepDate']=Map_Flt_To_DepTime['DepDay']+'/'+Map_Flt_To_DepTime['DepMonth']+'/'+Map_Flt_To_DepTime['DepYear']

all_FlightNumbers=Map_Flt_To_DepTime['FltNum'].unique().tolist()
Data=Data.loc[Data['FltNum'].apply(lambda x: x in all_FlightNumbers),:]
Data.index=range(Data.shape[0])

Map_Flt_To_DepTime['DepDayMonth']=Map_Flt_To_DepTime['DepDate'].apply(lambda x: x.split('/')[0]+'/'+x.split('/')[1])
Map_Flt_To_DepTime['DepMonth']=Map_Flt_To_DepTime['DepDate'].apply(lambda x: x.split('/')[1]+'/'+x.split('/')[2])

Map_Flt_To_DepTime=Map_Flt_To_DepTime[['FltNum','DepDate','DepDayMonth','DepMonth','DepTime']]

# DO THE MAPPING 

Data=Data.merge(Map_Flt_To_DepTime,on=['FltNum','DepDate'],how='left')
Data=Data[[x for x in Data.columns if '_y' not in x]]
Data.columns=[x.replace('_x','') for x in Data.columns]
Data=Data.rename(columns={'DepTime': 'DepTime_mostgran'})

Data=Data.merge(Map_Flt_To_DepTime.drop_duplicates(subset=['FltNum','DepDayMonth']),on=['FltNum','DepDayMonth'],how='left')
Data=Data[[x for x in Data.columns if '_y' not in x]]
Data.columns=[x.replace('_x','') for x in Data.columns]
Data=Data.rename(columns={'DepTime': 'DepTime_lessgran'})

Data=Data.merge(Map_Flt_To_DepTime.drop_duplicates(subset=['FltNum','DepMonth']),on=['FltNum','DepMonth'],how='left')
Data=Data[[x for x in Data.columns if '_y' not in x]]
Data.columns=[x.replace('_x','') for x in Data.columns]
Data=Data.rename(columns={'DepTime': 'DepTime_leastgran'})

Data['DepTime']=np.nan

Data.loc[~pd.isnull(Data['DepTime_mostgran']),'DepTime']=Data.loc[~pd.isnull(Data['DepTime_mostgran']),'DepTime_mostgran']
Data.loc[pd.isnull(Data['DepTime_mostgran']),'DepTime']=Data.loc[pd.isnull(Data['DepTime_mostgran']),'DepTime_lessgran']
Data.loc[pd.isnull(Data['DepTime_mostgran'])&pd.isnull(Data['DepTime_lessgran']),'DepTime']\
        =Data.loc[pd.isnull(Data['DepTime_mostgran'])&pd.isnull(Data['DepTime_lessgran']),'DepTime_leastgran']

Data=Data.loc[~pd.isnull(Data['DepTime']),:]

### Change format of remaining columns

In [100]:
Data=Data.loc[~pd.isnull(Data['DepTime']),['IssueDate','DepDate','FltNum','SegTo','SegFrom','Direction','DepTime','NumPax',
                                     'Coupons+YQ on current bookings','Current revenue','Group pax','Cabin Capacity']].copy()

IssueDate=Data['IssueDate'].unique().tolist()
IssueDate_Map=DataFrame([IssueDate,
[pd.to_datetime(x.split(' ')[0],format='%d/%m/%Y') for x in IssueDate]]).transpose()
IssueDate_Map.columns=['IssueDate','NewFormat']

Data=Data.merge(IssueDate_Map,on=['IssueDate'])
Data=Data[[x for x in Data.columns if x!='IssueDate']]
Data=Data.rename(columns={'NewFormat': 'IssueDate'})

#######################################

DepartureDate=Data['DepDate'].unique().tolist()
DepartureDate_Map=DataFrame([DepartureDate,
[pd.to_datetime(x.split(' ')[0],format='%d/%m/%Y') for x in DepartureDate]]).transpose()
DepartureDate_Map.columns=['DepDate','NewFormat']
DepartureDate_Map['yday']=DepartureDate_Map['NewFormat'].apply(lambda x: x.timetuple().tm_yday)

DepartureDate_Map['dday_num']=DepartureDate_Map['NewFormat'].apply(lambda x: x.weekday())
Weekday_Map=DataFrame([[x for x in range(0,8)],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]).transpose()
Weekday_Map.columns=['dday_num','dday']
Weekday_Map['dday_num']=Weekday_Map['dday_num'].astype('int')

DepartureDate_Map=DepartureDate_Map.merge(Weekday_Map,on='dday_num')

DepartureDate_Map=DepartureDate_Map[[x for x in DepartureDate_Map if x!='dday_num']]
DepartureDate_Map=DepartureDate_Map.sort_values(by='NewFormat')

Data=Data.merge(DepartureDate_Map,on=['DepDate'])
Data=Data[[x for x in Data.columns if x!='DepDate']]
Data=Data.rename(columns={'NewFormat': 'DepDate'})

#######################################

DepartureTime=Data['DepTime'].unique().tolist()
DepartureTime_Map=DataFrame([DepartureTime,[pd.to_datetime(x,format='%H:%M:%S') for x in DepartureTime]]).transpose()
DepartureTime_Map.columns=['DepTime','NewFormat']
DepartureTime_Map['dtime']=DepartureTime_Map['NewFormat'].apply(lambda x: (x-datetime.datetime.combine(x.date(),datetime.time(0,0))).seconds/60)

Data=Data.merge(DepartureTime_Map,on=['DepTime'])
Data=Data[[x for x in Data.columns if x!='DepTime']]
Data=Data.rename(columns={'NewFormat': 'DepTime'})

#######################################

Data['month']=Data['DepDate'].apply(lambda x: int(x.month))

#######################################

Data['Dprio']=(Data['DepDate']-Data['IssueDate']).apply(lambda x: float(x.days))
Data=Data.loc[(Data['Dprio']>0)&(Data['Dprio']<=366)&(Data['NumPax']>0)&(~pd.isnull(Data['DepTime'])),:]

Data=Data.sort_values(by=['DepDate','Dprio'],ascending=[True,False])

In [101]:
Data.loc[Data['DepDate']==pd.to_datetime('2019-01-01'),:]

,FltNum,SegTo,SegFrom,Direction,NumPax,Coupons+YQ on current bookings,Current revenue,Group pax,Cabin Capacity,IssueDate,DepDate,yday,dday,DepTime,dtime,month,Dprio
84395,9416,SCN,TXL,I,2.0,20,46,0,67,2018-04-28 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,248.0
84396,9416,SCN,TXL,I,2.0,20,46,0,67,2018-04-29 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,247.0
84397,9416,SCN,TXL,I,2.0,20,46,0,67,2018-04-30 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,246.0
84398,9416,SCN,TXL,I,2.0,20,46,0,67,2018-05-01 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,245.0
84399,9416,SCN,TXL,I,3.0,30,66,0,67,2018-05-02 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,244.0
84400,9416,SCN,TXL,I,3.0,30,66,0,67,2018-05-03 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,243.0
84401,9416,SCN,TXL,I,3.0,30,66,0,67,2018-05-04 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,242.0
84402,9416,SCN,TXL,I,3.0,30,66,0,67,2018-05-05 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,241.0
84403,9416,SCN,TXL,I,3.0,30,69,0,67,2018-05-06 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,240.0
84404,9416,SCN,TXL,I,3.0,30,69,0,67,2018-05-07 00:00:00,2019-01-01 00:00:00,1,Tuesday,1900-01-01 15:50:00,950.0,1,239.0


# Pax Data

In [102]:
Pax=Data[['DepDate','Dprio','yday','dtime','Direction','month','dday','NumPax']].copy()
Pax=Pax.set_index(['DepDate','Dprio','yday','dtime','Direction','month','dday'])

Pax=Pax.groupby(level=[0,2,3,4,5,6]).diff().reset_index()
Pax=Pax.dropna(how='any')
Data_NoObs=Pax.loc[:,['DepDate','yday','dtime','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['NumPax']=0

Data_NoObs=Data_NoObs[Pax.columns]

Pax=pd.concat([Pax,Data_NoObs])
Pax=Pax.drop_duplicates(subset=['DepDate','Dprio','yday','dtime','Direction','month','dday'])

Pax=Pax.sort_values(by=['DepDate','dtime','Dprio'],ascending=[True,True,False])
Pax.index=range(Pax.shape[0])

# Group bookings

In [103]:
GroupPax=Data[['DepDate','Dprio','yday','dtime','Direction','month','dday','Group pax']].copy()
GroupPax=GroupPax.set_index(['DepDate','Dprio','yday','dtime','Direction','month','dday'])

GroupPax=GroupPax.groupby(level=[0,2,3,4,5,6]).diff().reset_index()
GroupPax=GroupPax.dropna(how='any')
Data_NoObs=GroupPax.loc[:,['DepDate','yday','dtime','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['Group pax']=0

Data_NoObs=Data_NoObs[GroupPax.columns]

GroupPax=pd.concat([GroupPax,Data_NoObs])
GroupPax=GroupPax.drop_duplicates(subset=['DepDate','Dprio','yday','dtime','Direction','month','dday'])

GroupPax=GroupPax.sort_values(by=['DepDate','dtime','Dprio'],ascending=[True,True,False])
GroupPax.index=range(GroupPax.shape[0])

# Revenue 

In [104]:
Revenue=Data[['DepDate','Dprio','yday','dtime','Direction','month','dday','Coupons+YQ on current bookings']].copy()
Revenue=Revenue.set_index(['DepDate','Dprio','yday','dtime','Direction','month','dday'])

Revenue=Revenue.groupby(level=[0,2,3,4,5,6]).diff().reset_index()
Revenue=Revenue.dropna(how='any')
Data_NoObs=Revenue.loc[:,['DepDate','yday','dtime','Direction','month','dday']].drop_duplicates().copy()

distinct_flights=Data_NoObs.shape[0]
Data_NoObs.index=range(Data_NoObs.shape[0])
Data_NoObs=Data_NoObs.iloc[np.repeat([x for x in Data_NoObs.index],365),:]

Data_NoObs['Dprio']=[x for x in range(1,366)]*distinct_flights
Data_NoObs['Coupons+YQ on current bookings']=0

Data_NoObs=Data_NoObs[Revenue.columns]

Revenue=pd.concat([Revenue,Data_NoObs])
Revenue=Revenue.drop_duplicates(subset=['DepDate','Dprio','yday','dtime','Direction','month','dday'])

Revenue=Revenue.sort_values(by=['DepDate','dtime','Dprio'],ascending=[True,True,False])
Revenue.index=range(Revenue.shape[0])

# Data Export

In [105]:
Pax=Pax.merge(GroupPax,on=['DepDate','Dprio','yday','dtime','Direction','month','dday'])
Pax['NumPax']=Pax['NumPax']-Pax['Group pax']

TrainingSet=Pax.loc[(Pax['DepDate']>=pd.to_datetime('2018-04-01'))&(Pax['DepDate']<=pd.to_datetime('2019-03-20')),].copy()
TestSet=Pax.loc[Pax['DepDate']>=pd.to_datetime('2019-04-01'),].copy()

TrainingSet.to_csv('R_Training_Counting.csv',index=False)
TestSet.to_csv('R_Test_Counting.csv',index=False)

#################################################

GroupPax.to_csv('GroupPax.csv',index=False)

#################################################

Revenue.to_csv('Revenue.csv',index=False)